In [2]:
import os

os.environ["DGLBACKEND"] = "pytorch" # 配置DGL后端为PyTorch
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

d:\miniconda\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")
'''
2708个节点，10556条边，每个节点具有1433维的特征，共7个类别
mask分别标记了训练集、验证集和测试集的节点
'''

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


'\n2708个节点，10556条边，每个节点具有1433维的特征，共7个类别\nmask分别标记了训练集、验证集和测试集的节点\n'

In [4]:
g = dataset[0] # 一个dataset可能有多个图，这里只有一个图 DGLGraph对象


print("Node features")
print(g.ndata)
print("Edge features")
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


In [6]:
g.edges()[0], g.edges()[1]

(tensor([   0,    0,    0,  ..., 2707, 2707, 2707]),
 tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]))

In [5]:
# 用现成的图卷积模块
from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h) # 输出未归一化
        return h 


# Create the model with given dimensions
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

In [ ]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1) # 返回得分最大的索引

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )


model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.945, val acc: 0.132 (best 0.132), test acc: 0.133 (best 0.133)
In epoch 5, loss: 1.886, val acc: 0.406 (best 0.414), test acc: 0.435 (best 0.424)
In epoch 10, loss: 1.801, val acc: 0.476 (best 0.476), test acc: 0.486 (best 0.486)
In epoch 15, loss: 1.694, val acc: 0.576 (best 0.576), test acc: 0.589 (best 0.589)
In epoch 20, loss: 1.566, val acc: 0.618 (best 0.618), test acc: 0.628 (best 0.628)
In epoch 25, loss: 1.420, val acc: 0.662 (best 0.662), test acc: 0.679 (best 0.679)
In epoch 30, loss: 1.262, val acc: 0.684 (best 0.684), test acc: 0.715 (best 0.715)
In epoch 35, loss: 1.099, val acc: 0.708 (best 0.710), test acc: 0.731 (best 0.728)
In epoch 40, loss: 0.937, val acc: 0.726 (best 0.726), test acc: 0.742 (best 0.742)
In epoch 45, loss: 0.785, val acc: 0.734 (best 0.734), test acc: 0.753 (best 0.753)
In epoch 50, loss: 0.647, val acc: 0.740 (best 0.742), test acc: 0.761 (best 0.758)
In epoch 55, loss: 0.528, val acc: 0.754 (best 0.754), test acc: 0.765 (best 0